In [3]:
# data structuring
import pandas as pd
import numpy as np
#from SpectraFP import SpectraFP
from sklearn.model_selection import train_test_split
from tqdm import tqdm


# build-models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# avaluated models
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score,StratifiedKFold, LeaveOneOut, KFold
from pyADA import Smetrics, Similarity, ApplicabilityDomain



In [4]:
df = pd.read_csv('df_esteroids_MB_IN_AR.csv', sep = ';')
df

,Classe,[M]+,[M-15]+,[M-29]+,[M-90]+,[M-2·90]+,[M-3·90],[M-90-15]+,[M-2·90-15]+,[M-3·90-15],...,[M-140],[M-157],[M-144],m/z 103,m/z 129,m/z 143,m/z 169,m/z 244,m/z 218,m/z 231
0,2,3,3,0,0,0,0,3,3,0,...,0,0,0,0,1,1,2,0,0,0
1,0,3,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,3,1,0,0,0,0,0,0,0,...,1,0,0,0,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,6,2,1,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
154,6,3,0,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,3
155,6,3,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
156,6,1,2,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
# load Models
import pickle
model_RFC = pickle.load(open('rf_anabolizante_model.pkl', 'rb'))
model_XGBC = pickle.load(open('XGradientBoosting_anabolizante_model.pkl', 'rb'))
#model_NN = pickle.load(open('Class_steroids/model_NN_testo', 'rb'))


In [6]:
from collections import Counter

# Supondo que df seja o seu DataFrame

# Inicializando uma nova coluna 'Classe_bin' com valor padrão 0
df['Classe_AAS'] = 0

# Atribuindo valor 1 para o intervalo [91:,:]
df.loc[92:, 'Classe_AAS'] = 1

# Exibindo as contagens após a separação
counter = Counter(df['Classe_AAS'])
yi = df['Classe_AAS']
yi = np.array(yi)
yi


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [9]:
# train and test split
Xi = df.iloc[:,1:]


In [10]:
# balanceamento
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Verificando a contagem inicial das classes
print("Contagem inicial das classes:", Counter(yi))

# Aplicando UnderSampler usando RandomUnderSampler
rs = RandomUnderSampler(random_state=1)
xi_resampled, yi_resampled = rs.fit_resample(Xi, yi)

# Verificando a contagem após o UnderSampler
print("Contagem após UnderSampler:", Counter(yi_resampled))

Contagem inicial das classes: Counter({0: 92, 1: 66})
Contagem após UnderSampler: Counter({0: 66, 1: 66})


# Stratified K-fold

In [11]:
import scipy
Scaler = Smetrics()

In [13]:
from sklearn.model_selection import StratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(Xi,yi, random_state=29)
xi_ = X_train.copy()
xi_ = xi_.values
y_ = y_train.copy()
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
togetindexs = np.zeros(shape=(xi_.shape[0], 1))

# listas
## RF
mcc_RFC = []
kappa_RFC = []
accuracy_RFC = []

## XGB
mcc_GBC = []
kappa_GBC = []
accuracy_GBC = []

### MLP
#rmse_ext_mlp = []
#R_2ext_mlp = []
#Q_2_mlp = []
#r2_pearson_mlp = []
#score_mlp = [ ]

for i_train, i_test in tqdm(skfold.split(xi_, togetindexs)):
   
    # Random Forest
    model_RFC.fit(xi_[i_train],y_[i_train])
    y_pred_RFC = model_RFC.predict(xi_[i_test])
   
    # XGBoost
    model_XGBC.fit(xi_[i_train],y_[i_train])
    y_pred_GBC = model_XGBC.predict(xi_[i_test])
    
   ## mlpreg
   #model_NN.fit(xi_[i_train],y_[i_train])
   #y_pred_mlp =  model_NN.predict(xi_[i_test])
    

    ## Analise metricas Externas RF
    kappa_RFC.append(Scaler.kappaCalc(y_[i_test],y_pred_RFC))
    mcc_RFC.append(matthews_corrcoef(y_[i_test],y_pred_RFC))
    accuracy_RFC.append(accuracy_score(y_[i_test], y_pred_RFC))
    ## Analise metricas Externas XGB
    kappa_GBC.append(Scaler.kappaCalc(y_[i_test],y_pred_GBC))
    mcc_GBC.append(matthews_corrcoef(y_[i_test],y_pred_GBC))
    accuracy_GBC.append(accuracy_score(y_[i_test], y_pred_RFC))
    ## Analise metricas Externas MPL
    #rmse_ext_mlp.append(Scaler.roots_mean_square_error(y_[i_test],y_pred_mlp))
    #R_2ext_mlp.append(Scaler.R2ext(y_[i_test],y_pred_mlp))
    #Q_2_mlp.append(Scaler.Q2ext(y_[i_test],y_pred_mlp,y_[i_train]))
    #r2_pearson_mlp.append(scipy.stats.pearsonr(y_pred_mlp, y_[i_test]))
    #score_mlp.append(model_NN.score(xi_[i_train], y_[i_train]))

# table
dic = {'kappa_RF':kappa_RFC,'MCC_RF':mcc_RFC, 'Accuracy_RF':accuracy_RFC, 'kappa_GBC':kappa_GBC,'MCC_GBC':mcc_GBC, 'Accuracy_GBC':accuracy_GBC}
df_CV = pd.DataFrame(dic)
df_CV

10it [00:00, 53.48it/s]


,kappa_RF,MCC_RF,Accuracy_RF,kappa_GBC,MCC_GBC,Accuracy_GBC
0,1.000000,1.00000,1.000000,1.0,1.0,1.000000
1,1.000000,1.00000,1.000000,1.0,1.0,1.000000
2,0.823529,0.83666,0.916667,1.0,1.0,0.916667
3,1.000000,1.00000,1.000000,1.0,1.0,1.000000
4,1.000000,1.00000,1.000000,1.0,1.0,1.000000
5,1.000000,1.00000,1.000000,1.0,1.0,1.000000
6,1.000000,1.00000,1.000000,1.0,1.0,1.000000
7,1.000000,1.00000,1.000000,1.0,1.0,1.000000
8,1.000000,1.00000,1.000000,1.0,1.0,1.000000
9,1.000000,1.00000,1.000000,1.0,1.0,1.000000


In [12]:
df_CV['Accuracy_GBC'].mean()

0.7206140350877193

In [13]:
kappa_RFC

[0.30000000000000004,
 0.47916666666666663,
 0.28571428571428564,
 0.07692307692307697,
 0.3294117647058823,
 0.40624999999999994]

# Leave one out

In [37]:
# cross-validation Leave-one-out
loo = LeaveOneOut()
# score
score_RFC=cross_val_score(model_RFC,X,Y,cv=loo)
score_GBC=cross_val_score(model_GBC,X,Y,cv=loo)
print("Cross Validation Score: \n RFC: {rf}\n GBC: {gb}\n ".format(rf=score_RFC.mean(),gb=score_GBC.mean()))

Cross Validation Score: 
 RFC: 0.7278481012658228
 GBC: 0.6645569620253164
 


Cross Validation Score: 
 RFC: 0.7278481012658228
 GBC: 0.6645569620253164
 


In [18]:
pd.DataFrame(score_RFC).value_counts()

1.0    115
0.0     43
dtype: int64

In [25]:
loo = LeaveOneOut()
model_NB = GaussianNB()
model_SVC = SVC()
X_train, X_test, y_train, y_test = train_test_split(X,Y, random_state=13366)
xi_ = X_train.copy()
xi_ = xi_.values
y_ = y_train.copy()
togetindexs = np.zeros(shape=(xi_.shape[0], 1))

# listas
## RF
accuracy_RFC = []

## GB
mcc_GBC = []

accuracy_GBC = []

## NB
accuracy_NB = []

## SVC
accuracy_SVC = []


for i_train, i_test in tqdm(loo.split(xi_, togetindexs)):
   
    # Random Forest
    model_RFC.fit(xi_[i_train],y_[i_train])
    y_pred_RFC = model_RFC.predict(xi_[i_test])
   
    # GBoost
    model_GBC.fit(xi_[i_train],y_[i_train])
    y_pred_GBC = model_GBC.predict(xi_[i_test])
    
    # Naive_bayes
    model_NB.fit(xi_[i_train],y_[i_train])
    y_pred_NB = model_NB.predict(xi_[i_test])
    
    # SVM
    model_SVC.fit(xi_[i_train],y_[i_train])
    y_pred_SVC = model_SVC.predict(xi_[i_test])

    ## Analise metricas Externas RF
    accuracy_RFC.append(accuracy_score(y_[i_test], y_pred_RFC))
    
    ## Analise metricas Externas GB
    accuracy_GBC.append(accuracy_score(y_[i_test], y_pred_GBC))
    
    ## Analise metricas Externas NB
    accuracy_NB.append(accuracy_score(y_[i_test], y_pred_NB))
    
    ## Analise metricas Externas SVC
    accuracy_SVC.append(accuracy_score(y_[i_test], y_pred_SVC))
    

# table
dic = {'Accuracy_RF':accuracy_RFC , 'Accuracy_GB':accuracy_GBC, "Accuracy_NB":accuracy_NB, "Accuracy_SVC":accuracy_SVC}
df_CV = pd.DataFrame(dic)
df_CV.loc['Accuracy']=[df_CV['Accuracy_RF'].mean(),df_CV['Accuracy_GB'].mean(),df_CV['Accuracy_NB'].mean(),df_CV['Accuracy_SVC'].mean()]
df_CV

118it [00:14,  8.36it/s]


,Accuracy_RF,Accuracy_GB,Accuracy_NB,Accuracy_SVC
0,1.000000,1.000000,0.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000
2,0.000000,0.000000,0.000000,0.000000
3,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...
114,1.000000,1.000000,0.000000,1.000000
115,1.000000,1.000000,0.000000,1.000000
116,1.000000,1.000000,1.000000,1.000000
117,0.000000,1.000000,0.000000,1.000000


In [18]:
df_CV

,Accuracy_RF,Accuracy_GB
0,1.000000,1.000000
1,1.000000,1.000000
2,0.000000,0.000000
3,1.000000,1.000000
4,1.000000,1.000000
...,...,...
114,1.000000,1.000000
115,1.000000,1.000000
116,1.000000,1.000000
117,0.000000,1.000000
